## Objective Function:

Maximizing the revenue, given as the difference between the reward (profit) and the cost per click times the clicks ( costs ). 

It can be expressed as :
### $ max_{price,bid}( \sum \limits_{C_i} (price \times soldItems(bid, price)) - (clicks(bid) \times costPerClick(bid))) $


